In [93]:
# %matplotlib inline
import numpy as np
import pandas as pd
import glob
import os
import sys
import calendar
import datetime
import time
import matplotlib.pyplot as plt
from sklearn import datasets
from sklearn import metrics
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn import neighbors
from sklearn.neighbors import KNeighborsClassifier
from sklearn import svm
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import VotingClassifier

import azureml.core
from azureml.core import Workspace

# # check core SDK version number
# print("Azure ML SDK Version: ", azureml.core.VERSION)

In [94]:
# load workspace configuration from the config.json file in the current folder.
ws = Workspace.from_config()
print(ws.name, ws.location, ws.resource_group, ws.location, sep = '\t')

myworkspace	southeastasia	shrg	southeastasia


In [95]:
experiment_name = 'Stay_Home'

from azureml.core import Experiment
exp = Experiment(workspace=ws, name=experiment_name)

In [96]:
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
import os

# choose a name for your cluster
compute_name = os.environ.get("AML_COMPUTE_CLUSTER_NAME", "cpucluster")
compute_min_nodes = os.environ.get("AML_COMPUTE_CLUSTER_MIN_NODES", 0)
compute_max_nodes = os.environ.get("AML_COMPUTE_CLUSTER_MAX_NODES", 4)

# This example uses CPU VM. For using GPU VM, set SKU to STANDARD_NC6
vm_size = os.environ.get("AML_COMPUTE_CLUSTER_SKU", "STANDARD_D2_V2")


if compute_name in ws.compute_targets:
    compute_target = ws.compute_targets[compute_name]
    if compute_target and type(compute_target) is AmlCompute:
        print('found compute target. just use it. ' + compute_name)
else:
    print('creating a new compute target...')
    provisioning_config = AmlCompute.provisioning_configuration(vm_size = vm_size,
                                                                min_nodes = compute_min_nodes, 
                                                                max_nodes = compute_max_nodes)

    # create the cluster
    compute_target = ComputeTarget.create(ws, compute_name, provisioning_config)
    
    # can poll for a minimum number of nodes and for a specific timeout. 
    # if no min node count is provided it will use the scale settings for the cluster
    compute_target.wait_for_completion(show_output=True, min_node_count=None, timeout_in_minutes=20)
    
     # For a more detailed view of current AmlCompute status, use get_status()
    print(compute_target.get_status().serialize())

found compute target. just use it. cpucluster


In [97]:
import urllib.request
import os

data_folder = os.path.join(os.getcwd(), 'data')
os.makedirs(data_folder, exist_ok = True)

# urllib.request.urlretrieve('http://yann.lecun.com/exdb/mnist/train-images-idx3-ubyte.gz', filename=os.path.join(data_folder, 'train-images.gz'))
# urllib.request.urlretrieve('http://yann.lecun.com/exdb/mnist/train-labels-idx1-ubyte.gz', filename=os.path.join(data_folder, 'train-labels.gz'))
# urllib.request.urlretrieve('http://yann.lecun.com/exdb/mnist/t10k-images-idx3-ubyte.gz', filename=os.path.join(data_folder, 'test-images.gz'))
# urllib.request.urlretrieve('http://yann.lecun.com/exdb/mnist/t10k-labels-idx1-ubyte.gz', filename=os.path.join(data_folder, 'test-labels.gz'))

In [98]:
# make sure utils.py is in the same directory as this code
from utils import load_data

# Data import
# df=pd.read_csv('subway_merge_1.csv', encoding = 'utf-8')
data=pd.read_csv("Cloud&UV08-17 danger25.csv", engine='python', header=0).dropna()
# data2=pd.read_csv("highway.csv", engine='python', header=0)
# data3=pd.read_csv("THSeoulCrime.csv", engine='python', header=0)

x=data.iloc[:,:3]
y=data.iloc[:,-1:].values.reshape(-1,1)
xtrain,xtest,ytrain,ytest=train_test_split(x,y,test_size=0.3,random_state=0)

# x2=data2.iloc[:,:1]
# y2=data2.iloc[:,-1:].values.reshape(-1,1)
# xtrain2,xtest2,ytrain2,ytest2=train_test_split(x2,y2,test_size=0.3,random_state=0)

# x3=data3['Month'].values.reshape(-1,1)
# y3=data3['Crime.occur'].values.reshape(-1,1)
# xtrain3,xtest3,ytrain3,ytest3=train_test_split(x3,y3,test_size=0.3,random_state=0)

# x1=data.values[:,:2]
# y1=data.values[:,-1:]
# plt.figure()
# plt.ylabel("day")
# plt.xlabel("month")
# plt.scatter(data['월'],data['일'],c=data['자외선위험'],alpha=0.5)
# plt.show()

# x2_1=data2['time'].values.reshape(-1,1)
# y2_1=data2['hacc.ratio'].values.reshape(-1,1)
# xtrain2_1,xtest2_1,ytrain2_1,ytest2_1=train_test_split(x2_1,y2_1,test_size=0.3,random_state=0)
# plt.scatter(xtrain2_1,ytrain2_1)
# plt.show()

In [27]:
# # note we also shrink the intensity values (X) from 0-255 to 0-1. This helps the model converge faster.
# X_train = load_data(os.path.join(data_folder, 'train-images.gz'), False) / 255.0
# X_test = load_data(os.path.join(data_folder, 'test-images.gz'), False) / 255.0
# y_train = load_data(os.path.join(data_folder, 'train-labels.gz'), True).reshape(-1)
# y_test = load_data(os.path.join(data_folder, 'test-labels.gz'), True).reshape(-1)

# # now let's show some randomly chosen images from the traininng set.
# count = 0
# sample_size = 30
# plt.figure(figsize = (16, 6))
# for i in np.random.permutation(X_train.shape[0])[:sample_size]:
#     count = count + 1
#     plt.subplot(1, sample_size, count)
#     plt.axhline('')
#     plt.axvline('')
#     plt.text(x=10, y=-10, s=y_train[i], fontsize=18)
#     plt.imshow(X_train[i].reshape(28, 28), cmap=plt.cm.Greys)
# plt.show()



# ds = ws.get_default_datastore()
# print(ds.datastore_type, ds.account_name, ds.container_name)

# ds.upload(src_dir=data_folder, target_path='SH', overwrite=True, show_progress=True)

AzureBlob myworkspstorage66603882c azureml-blobstore-750332db-0d0f-4fd6-9d39-b0214037ef7c
Uploading C:\Users\Administrator\Documents\GitHub\First-Step\Test\data\test-images.gz
Uploading C:\Users\Administrator\Documents\GitHub\First-Step\Test\data\test-labels.gz
Uploading C:\Users\Administrator\Documents\GitHub\First-Step\Test\data\train-images.gz
Uploading C:\Users\Administrator\Documents\GitHub\First-Step\Test\data\train-labels.gz
Uploaded C:\Users\Administrator\Documents\GitHub\First-Step\Test\data\test-labels.gz, 1 files out of an estimated total of 4
Uploaded C:\Users\Administrator\Documents\GitHub\First-Step\Test\data\train-labels.gz, 2 files out of an estimated total of 4
Uploaded C:\Users\Administrator\Documents\GitHub\First-Step\Test\data\test-images.gz, 3 files out of an estimated total of 4
Uploaded C:\Users\Administrator\Documents\GitHub\First-Step\Test\data\train-images.gz, 4 files out of an estimated total of 4


$AZUREML_DATAREFERENCE_06ae8a84d57544f1a82466eab0631909

In [99]:
import os
script_folder  = os.path.join(os.getcwd(), "Stay_Home_script")
os.makedirs(script_folder, exist_ok=True)

In [100]:
%%writefile $script_folder/train.py

import argparse
import os
import numpy as np
import pandas as pd
import glob
import sys
import calendar
import datetime
import time
import matplotlib.pyplot as plt
from sklearn import datasets
from sklearn import metrics
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn import neighbors
from sklearn.neighbors import KNeighborsClassifier
from sklearn import svm
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.externals import joblib

import azureml.core
from azureml.core import Workspace

from azureml.core import Run
from utils import load_data

# let user feed in 2 parameters, the location of the data files (from datastore), and the regularization rate of the logistic regression model
# parser = argparse.ArgumentParser()
# parser.add_argument('--data-folder', type=str, dest='data_folder', help='data folder mounting point')
# parser.add_argument('--regularization', type=float, dest='reg', default=0.01, help='regularization rate')
# args = parser.parse_args()

# data_folder = args.data_folder
# print('Data folder:', data_folder)
data=pd.read_csv("Cloud&UV08-17 danger25.csv", engine='python', header=0).dropna()
# data2=pd.read_csv("highway.csv", engine='python', header=0)
# data3=pd.read_csv("THSeoulCrime.csv", engine='python', header=0)

# load train and test set into numpy arrays
# note we scale the pixel intensity values to 0-1 (by dividing it with 255.0) so the model can converge faster.
x=data.iloc[:,:3]
y=data.iloc[:,-1:].values.reshape(-1,1)
xtrain,xtest,ytrain,ytest=train_test_split(x,y,test_size=0.3,random_state=0)

# x2=data2.iloc[:,:1]
# y2=data2.iloc[:,-1:].values.reshape(-1,1)
# xtrain2,xtest2,ytrain2,ytest2=train_test_split(x2,y2,test_size=0.3,random_state=0)

# x3=data3['Month'].values.reshape(-1,1)
# y3=data3['Crime.occur'].values.reshape(-1,1)
# xtrain3,xtest3,ytrain3,ytest3=train_test_split(x3,y3,test_size=0.3,random_state=0)

# y=int(input("년도:"))
# m=int(input("월:"))
# d=int(input("일:"))
# h=int(input("시간:"))
# today=datetime.date(y,m,d).isocalendar()[1:]
# np.where(obj2["주차요일"] == today)
# select_indices = list(np.where(obj2["주차요일"] == today)[0])
# top20=obj2.iloc[select_indices]
# SUBWAY_Congestion=top20.head(20)

# get hold of the current run
run = Run.get_context()

# print('Train a logistic regression model with regularization rate of', args.reg)
# K=3
# KNN=KNeighborsClassifier(K, weights='distance')
# KNN.fit(xtrain,ytrain)
LR = LogisticRegression()
LR.fit(xtrain, ytrain)
# KNN.fit(xtrain2,ytrain2)
# KNN.fit(xtrain3,ytrain3)

print('Predict the test set')
# KResult=KNN.predict(xtest)
LResult=LR.predict(xtest)
# KResult2=KNN.predict(xtest2)
# KResult3=KNN.predict(xtest3)

# calculate accuracy on the prediction
Accuracy=metrics.accuracy_score(ytest,LResult)
# Accuracy2=metrics.accuracy_score(ytest2,KResult2)
# Accuracy3=metrics.accuracy_score(ytest3,KResult3)
print('Accuracy is', Accuracy)
# print('Accuracy is', Accuracy2)
# print('Accuracy is', Accuracy3)

# predict
# KResult_1=KNN.predict([[m,d,0]])
# KResult2_2=KNN.predict([[h]])
# KResult3_2=KNN.predict([[m]])

# run.log('regularization rate', np.float(args.reg))
run.log('accuracy', np.float(Accuracy))

# os.makedirs('outputs', exist_ok=True)
# note file saved in the outputs folder is automatically uploaded into experiment record
joblib.dump(value=LR, filename='outputs/Stay_Home_model.pkl')

# # note we scale the pixel intensity values to 0-1 (by dividing it with 255.0) so the model can converge faster.
# X_train = load_data(os.path.join(data_folder, 'train-images.gz'), False) / 255.0
# X_test = load_data(os.path.join(data_folder, 'test-images.gz'), False) / 255.0
# y_train = load_data(os.path.join(data_folder, 'train-labels.gz'), True).reshape(-1)
# y_test = load_data(os.path.join(data_folder, 'test-labels.gz'), True).reshape(-1)
# print(X_train.shape, y_train.shape, X_test.shape, y_test.shape, sep = '\n')

# # get hold of the current run
# run = Run.get_context()

# print('Train a logistic regression model with regularization rate of', args.reg)
# clf = LogisticRegression(C=1.0/args.reg, random_state=42)
# clf.fit(X_train, y_train)

# print('Predict the test set')
# y_hat = clf.predict(X_test)

# # calculate accuracy on the prediction
# acc = np.average(y_hat == y_test)
# print('Accuracy is', acc)

# run.log('regularization rate', np.float(args.reg))
# run.log('accuracy', np.float(acc))

# os.makedirs('outputs', exist_ok=True)
# # note file saved in the outputs folder is automatically uploaded into experiment record
# joblib.dump(value=clf, filename='outputs/sklearn_mnist_model.pkl')

Overwriting C:\Users\Administrator\Documents\GitHub\First-Step\Test\Stay_Home_script/train.py


In [101]:
import shutil
shutil.copy('utils.py', script_folder)

'C:\\Users\\Administrator\\Documents\\GitHub\\First-Step\\Test\\Stay_Home_script\\utils.py'

In [102]:
from azureml.train.estimator import Estimator

# script_params = {
#     '--data-folder': ds.path('SH').as_mount(),
#     '--regularization': 0.8
# }

est = Estimator(source_directory=script_folder,
#                 script_params=script_params,
                compute_target=compute_target,
                entry_script='train.py',
                conda_packages=['scikit-learn'])

In [103]:
run = exp.submit(config=est)
run

Experiment,Id,Type,Status,Details Page,Docs Page
Stay_Home,Stay_Home_1558486519_b85fce0a,azureml.scriptrun,Queued,Link to Azure Portal,Link to Documentation


In [104]:
from azureml.widgets import RunDetails
RunDetails(run).show()

_UserRunWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', '…

In [105]:
run.wait_for_completion(show_output=False) # specify True for a verbose log

ActivityFailedException: Activity Failed:
{
    "error": {
        "code": "UserError",
        "message": "No module named 'pandas'",
        "details": [],
        "debugInfo": {
            "type": "ModuleNotFoundError",
            "message": "No module named 'pandas'",
            "stackTrace": "  File \"azureml-setup/context_manager_injector.py\", line 95, in execute_with_context\n    runpy.run_path(sys.argv[0], globals(), run_name=\"__main__\")\n  File \"/azureml-envs/azureml_088906b35cdd6596b350e033526a1806/lib/python3.6/runpy.py\", line 263, in run_path\n    pkg_name=pkg_name, script_name=fname)\n  File \"/azureml-envs/azureml_088906b35cdd6596b350e033526a1806/lib/python3.6/runpy.py\", line 96, in _run_module_code\n    mod_name, mod_spec, pkg_name, script_name)\n  File \"/azureml-envs/azureml_088906b35cdd6596b350e033526a1806/lib/python3.6/runpy.py\", line 85, in _run_code\n    exec(code, run_globals)\n  File \"train.py\", line 5, in <module>\n    import pandas as pd\n"
        

In [35]:
print(run.get_metrics())

{}


In [18]:
print(run.get_file_names())

['azureml-logs/55_batchai_execution.txt', 'azureml-logs/60_control_log.txt', 'azureml-logs/80_driver_log.txt', 'azureml-logs/azureml.log', 'outputs/sklearn_mnist_model.pkl']


In [19]:
# register model 
model = run.register_model(model_name='sklearn_mnist', model_path='outputs/sklearn_mnist_model.pkl')
print(model.name, model.id, model.version, sep = '\t')

sklearn_mnist	sklearn_mnist:1	1
